In [11]:
import pandas as pd
import numpy as np
cantidad_null = 6
dataframe = pd.read_csv("developers_info.csv",encoding="latin-1",sep=",")

col_no_eliminar = ["Employment","Country","YearsCode","Gender","Ethnicity","ConvertedCompYearly"]
TotalFilas = len(dataframe)
sr_contar_no_nulos = dataframe.count()
# el count devuelve una serie con núm de elementos no nulos ni NaN de c/col, si lo divido x total de filas y le resto 1, tengo los elementos nulos
sr_porcentaje = (1-(sr_contar_no_nulos/TotalFilas))*100

df = dataframe.fillna("Sin información")
# Paso por toda la serie y cuando cumple la condicion, elimino la columna, el index de la serie son los nombres de las col de df
for i, valor in enumerate(sr_porcentaje):
    if valor > cantidad_null:
        if not sr_porcentaje.index[i] in col_no_eliminar: # valido que nunca va a eliminar las columnas que serán ocupadas para selecciones posteriores.
            col = sr_porcentaje.index[i]
            del df[col]
df.columns

Index(['Employment', 'Country', 'EdLevel', 'LearnCode', 'YearsCode', 'DevType',
       'LanguageHaveWorkedWith', 'Age', 'Gender', 'Ethnicity',
       'ConvertedCompYearly'],
      dtype='object')

In [228]:
Paises = ["Germany", "Spain", "Chile", "Austria", "United States of America"]
df.drop(df[df["ConvertedCompYearly"]=="Sin información"].index, inplace=True)
df['ConvertedCompYearly'] = df['ConvertedCompYearly'].astype('float64')

#df_paises = df.pivot_table(index="Country", values="ConvertedCompYearly", aggfunc=np.mean)
df_Tpaises = df.groupby("Country").agg({"ConvertedCompYearly":"mean"})
df_Tpaises.reset_index(inplace=True)
f = pd.DataFrame()
for pais in Paises:
    p = df_Tpaises[df_Tpaises["Country"]==pais]
    f = pd.concat([p,f],ignore_index=True)
    ##f.append(p,ignore_index=True) --- no puede usar el append pq me sale un error pq esta descontinuado 

f.sort_values(by="ConvertedCompYearly", ascending=False, inplace=True)
f["ConvertedCompYearly"]=f["ConvertedCompYearly"].map("{:,.0f}".format).apply(lambda x:x.replace(",","."))
f.rename(columns={"Country":"País", "ConvertedCompYearly":"Ingreso promedio"}, inplace=True)
f.loc[f["País"]=="United States of America", "País"]="USA"
f

,País,Ingreso promedio
0,USA,263.279
4,Germany,100.835
3,Spain,77.489
1,Austria,60.995
2,Chile,35.356


In [4]:
from colorama import Fore, Back, Style
print(Fore.RED + 'some red text')
print(Back.GREEN + 'and with a green background')
print(Style.DIM + 'and in dim text')
print(Style.RESET_ALL)
print('back to normal now')

some red text
and with a green background
and in dim text

back to normal now


In [35]:
import pandas as pd
import numpy as np
cantidad_null = 60
dataframe = pd.read_csv("developers_info.csv",encoding="latin-1",sep=",")
###################
col_no_eliminar = ["Employment","Country","YearsCode","Gender","Ethnicity","ConvertedCompYearly"]
TotalFilas = len(dataframe)
sr_contar_no_nulos = dataframe.count()
# el count devuelve una serie con núm de elementos no nulos ni NaN de c/col, si lo divido x total de filas y le resto 1, tengo los elementos nulos
sr_porcentaje = (1-(sr_contar_no_nulos/TotalFilas))*100
df = dataframe.fillna("Sin información")
# Paso por toda la serie y cuando cumple la condicion, elimino la columna, el index de la serie son los nombres de las col de df
for i, valor in enumerate(sr_porcentaje):
    if valor > cantidad_null:
        if not sr_porcentaje.index[i] in col_no_eliminar: # valido que nunca va a eliminar las columnas que serán ocupadas para selecciones posteriores.
            col = sr_porcentaje.index[i]
            del df[col]
###################
df = df.loc[:, ["YearsCode", "ConvertedCompYearly"]]   

df.drop(df[df["ConvertedCompYearly"]=="Sin información"].index, inplace=True)
df.drop(df[df["YearsCode"]=="Sin información"].index, inplace=True)

df["YearsCode"] = df["YearsCode"].str.replace("Less than 1 year", "0")
df["YearsCode"] = df["YearsCode"].str.replace("More than 50 years", "51")

df = df.astype({"YearsCode":float, "ConvertedCompYearly":float})  

df["Rango Edad"] = pd.cut(df["YearsCode"], bins=range(0, 60, 5), right=False)
df["Años de experiencia"] = df["Rango Edad"].apply(lambda x: f"{int(x.left)} - {int(x.right)}")

df2 = df.groupby("Años de experiencia", observed=False).agg({"ConvertedCompYearly":["min","max"]})

# Renombrar el nombre de las columnas
df2.columns = ["Intervalo inferior (USD/Año)","Intervalo superior (USD/Año)" ]

#FORMATEAR A MILES: usé la función map() que es como un for dentro del df, y la función anómima lambda, para realizar un replace()
df2["Intervalo inferior (USD/Año)"] = df2["Intervalo inferior (USD/Año)"].map("{:,.0f}".format).apply(lambda x:x.replace(",","."))
df2["Intervalo superior (USD/Año)"] = df2["Intervalo superior (USD/Año)"].map("{:,.0f}".format).apply(lambda x:x.replace(",","."))

# reintegro el índice, para que los Años de experiencia sean la primera columna. 
df2.reset_index(inplace=True)


df2

,Años de experiencia,Intervalo inferior (USD/Año),Intervalo superior (USD/Año)
0,0 - 5,4,6.500.000
1,5 - 10,2,30.468.516
2,10 - 15,2,20.000.000
3,15 - 20,6,9.000.000
4,20 - 25,3,12.500.000
5,25 - 30,1,17.500.000
6,30 - 35,1,10.000.000
7,35 - 40,103,21.822.250
8,40 - 45,180,1.608.000
9,45 - 50,400,5.000.000
